***

<font color='green'>

# Project Solution: GL Chatbot - Part 2

### Context: Great Learning is looking forward to design an automation which can interact with the user, understand the problem and display the resolution procedure [if found as a generic request] or redirect the request to an actual human support executive if the request is complex or not in it’s database. This is to support Academic Support department of Great Learning.

### Objective: Design a Python based interactive semi-rule based chatbot which can do the following:
1. Start chat session with greetings and ask what the user is looking for.
2. Accept dynamic text based questions from the user. Reply back with relevant answer from the designed corpus.
3. End the chat session only if the user requests to end else ask what the user is looking for. Loop continues till the user asks to end it.
    
### Section 2: This module contains the implementation of chatbot using the model built in Section 1
#### Input:
- GL Bot.json containing a designed corpus with tags, patterns, responses
- Model architecture and weigths (chatbotModel.json and chatbotModel.h5)

#### Usage:
- Please execute chat() function to start conversation with virtual agent.
    
***
***
*Prepared by: Sauvik De*

*Date: September 4, 2021*

</font>

##### Reference: https://www.mygreatlearning.com/blog/basics-of-building-an-artificial-intelligence-chatbot
***

In [1]:
import nltk, json
from tensorflow.keras.models import model_from_json

In [2]:
# Load model data
with open('GL Bot.json') as f:
    corpus = json.load(f)

In [3]:
# load json and model
json_file = open('chatbotModel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

# load weights into new model
model.load_weights("chatbotModel.h5")
print("Loaded model from disk")

Loaded model from disk


In [4]:
# Extract data
WORDS = []  #Tokens
LABELS = [] #Identified tags or labels

for intents in corpus['intents']:
    for pattern in intents['patterns']:
        w = nltk.word_tokenize(pattern)
        WORDS.extend(w)  #Bag of Words

    # Add the missing tag if any
    if intents['tag'] not in LABELS:
        LABELS.append(intents['tag'])

In [5]:
lemmatizer = nltk.stem.WordNetLemmatizer()
tag_dict = {"J": 'a', #nltk.corpus.wordnet.ADJ,
            "N": 'n', #nltk.corpus.wordnet.NOUN,
            "V": 'v', #nltk.corpus.wordnet.VERB,
            "R": 'r'  #nltk.corpus.wordnet.ADV
           }

def get_wordnet_pos(word):
   pTag = nltk.pos_tag([word])[0][1][0].upper()
   return tag_dict.get(pTag, 'n') 

WORDS = [lemmatizer.lemmatize(w, pos=get_wordnet_pos(w)).lower() for w in WORDS if w != '?']
WORDS = sorted(list(set(WORDS))) #Fetch unique list of words

In [6]:
def ohe_input(input_sent, W):
    bow = []
    for w in W:
        if w in input_sent:
            bow.append(1)
        else: bow.append(0)
    return bow

In [7]:
# Text chat utility function
def chat():
    import numpy as np
    
    print('Hello, my name is GLBOT\nI am your virtual assistant.')
    print('\n***To quit, please type: QUIT')
    print("***Not the response you are looking for? .. please type: *")
    
    while True:
        inputText = input('\n\nYou: ')
        if inputText.lower()=='*':
            print('GLBOT: Please rephrase your question and type again.')
            continue
        if inputText.lower()=='quit':
            print('GLBOT: Good Bye!')
            break
        
        results = model.predict([ohe_input(inputText.lower(), WORDS)])
        results_index = np.argmax(results)
        tag = LABELS[results_index]    
        
        for intent in corpus['intents']:
            if intent['tag'] == tag:
                responses = intent['responses']
                print('GLBOT:', responses[0])
        
        #if tag == 'Exit':
        #    break

In [8]:
chat()

Hello, my name is GLBOT
I am your virtual assistant.

***To quit, please type: QUIT
***Not the response you are looking for? .. please type: *


You: hi need help pls
GLBOT: Hello! how may I help you?


You: what is machine learning
GLBOT: Link: Machine Learning wiki 


You: deep learning concepts?
GLBOT: Link: Neural Nets wiki


You: what the hell!
GLBOT: Please use respectful words


You: *
GLBOT: Please rephrase your question and type again.


You: i think i am good for now, thanks
GLBOT: I hope I was able to assist you. Please type in Quit to exit. Good Bye!


You: quit
GLBOT: Good Bye!
